In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
app_root = os.path.abspath(os.path.join(project_root, '../../app', '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    sys.path.append(app_root)

In [2]:
from train.datasets_preprocessing.datasets_preprocessing import load_json_data, make_pipeline
import pandas as pd

math_pipeline = make_pipeline('math')

X_json_raw = load_json_data('../datasets_preprocessing/datasets/math')
math_pipeline.fit_transform(X_json_raw)

math_df = pd.read_csv(os.path.join('../datasets_preprocessing/csv_question_files', 'math.csv'))
math_df.head(10)

,question,tags_str,math,bio,code
0,Function on the unit circle and exponential,"complex-analysis,continuity",1,0,0
1,What does $\sum_{n=0}^\infty 1/n^n$ converge to?,"sequences-and-series,number-theory,limits",1,0,0
2,Random variable measurable with respect to sto...,"measure-theory,random-variables,stopping-times...",1,0,0
3,What is the distribution of 2 consecutive Bino...,"probability,binomial-coefficients,binomial-dis...",1,0,0
4,Determine lines intersecting four skew lines i...,"projective-geometry,projective-space,cross-ratio",1,0,0
5,How do you deal with absolute values in a func...,"calculus,solid-of-revolution",1,0,0
6,Aren't $ f’(xy) $ and $ f’(x/y)$ ambiguous not...,"multivariable-calculus,functions",1,0,0
7,Why do counits go that way?,"soft-question,category-theory,education,adjoin...",1,0,0
8,Not understanding a proof about coherent sheav...,"algebraic-geometry,proof-explanation,schemes,s...",1,0,0
9,Model theory of the naturals with a multiplica...,"model-theory,first-order-logic,nonstandard-models",1,0,0


In [3]:
bio_pipeline = make_pipeline('bio')

X_json_raw = load_json_data('../datasets_preprocessing/datasets/bio')
bio_pipeline.fit_transform(X_json_raw)

bio_df = pd.read_csv(os.path.join('../datasets_preprocessing/csv_question_files', 'bio.csv'))
bio_df.head(10)

,question,tags_str,math,bio,code
0,How many kg of seed can one expect from 230 kg...,agriculture,0,1,0
1,Adaptive Optics in Microscopy: what are the fa...,"biophysics,microscopy,fluorescent-microscopy,o...",0,1,0
2,Which part of the reflex arc takes the longest...,"human-biology,reflexes",0,1,0
3,Is wiping with RNAse Zap enough to destroy RNA...,"molecular-biology,lab-techniques,rna,lab-reagents",0,1,0
4,When there is incomplete dominance of one alle...,"genetics,terminology",0,1,0
5,Does drinking dry water have same effect as dr...,"human-biology,food",0,1,0
6,Do non-migratory canada geese still exhibit mi...,"ornithology,migration",0,1,0
7,"If life is discovered on another planet, will ...","taxonomy,astrobiology",0,1,0
8,Why do toenails grow much slower than fingerna...,"human-biology,human-anatomy",0,1,0
9,"Why do, humans, like many birds,tend to stand ...","brain,muscles,balance",0,1,0


In [4]:
code_pipeline = make_pipeline('code')

X_json_raw = load_json_data('../datasets_preprocessing/datasets/code')
code_pipeline.fit_transform(X_json_raw)

code_df = pd.read_csv(os.path.join('../datasets_preprocessing/csv_question_files', 'code.csv'))
code_df.head(10)

,question,tags_str,math,bio,code
0,WorkGroup Data Service with JSON / Web based API,"c#,linq,json",0,0,1
1,Design pattern for logger implementation,"object-oriented,design-patterns,salesforce-apex",0,0,1
2,Temperature calculator in Rust,"beginner,rust,unit-conversion",0,0,1
3,Read binary serial data and parse integers,"c#,serial-port",0,0,1
4,Brain-flak interpreter,"parsing,go,interpreter",0,0,1
5,Implement bash auto completion in Python,"python,python-3.x,bash,autocomplete",0,0,1
6,How well or poorly structured are my routes in...,"javascript,node.js",0,0,1
7,“Proper” Asynchronous implementation,"c#,asynchronous",0,0,1
8,Determining whether a loop iterated at least o...,"python,python-3.x,generator",0,0,1
9,Output JavaScript object into HTML table of ke...,"javascript,jquery",0,0,1


In [5]:
full_df = pd.concat(
	[
		math_df,
		bio_df,
		code_df
	],
    ignore_index=True,
	axis=0
)

full_df = full_df.drop_duplicates(subset=["question"], keep="first")
full_df

,question,tags_str,math,bio,code
0,Function on the unit circle and exponential,"complex-analysis,continuity",1,0,0
1,What does $\sum_{n=0}^\infty 1/n^n$ converge to?,"sequences-and-series,number-theory,limits",1,0,0
2,Random variable measurable with respect to sto...,"measure-theory,random-variables,stopping-times...",1,0,0
3,What is the distribution of 2 consecutive Bino...,"probability,binomial-coefficients,binomial-dis...",1,0,0
4,Determine lines intersecting four skew lines i...,"projective-geometry,projective-space,cross-ratio",1,0,0
...,...,...,...,...,...
1445782,Virtual Memory - Non-contiguous Memory Allocation,"operating-systems,memory-management,virtual-me...",0,0,1
1445783,Residence time in multi server system,"algorithm-analysis,distributed-systems,queuein...",0,0,1
1445784,How can I improve my algorithm for finding opt...,"algorithms,partitions",0,0,1
1445785,When can I use dynamic programming to reduce t...,"algorithms,dynamic-programming,efficiency,algo...",0,0,1


In [6]:
from train.reporting.model_interface import ModelInterface
from train.reporting.text_svm_wrapper import TextSVMWrapper
from typing import Tuple
import pickle

import pandas as pd

def import_model_and_its_test_set(path: str) -> Tuple[ModelInterface, pd.DataFrame]:
        with open(path + "/model.pkl", "rb") as f:
            model = pickle.load(f)


        test_set = pd.read_csv(
            path + "/test_set.csv", index_col=0)
        return model, test_set



math_model, _ = import_model_and_its_test_set("saved_models/math")
bio_model, _ = import_model_and_its_test_set("saved_models/bio")
code_model, _ = import_model_and_its_test_set("saved_models/code")

# Tutaj wybieramy numer zbioru testowego

In [67]:
test_set_number = 2 # allowed 0 1 2

In [68]:
test_df = pd.read_csv(f"../datasets_preprocessing/datasets/test_all_models/test_{test_set_number}.csv", index_col=0)

test_df.drop_duplicates(subset=["question"], keep="first")
test_df

,question,tags_str,math,bio,code
949051,$(x + y)^s \le x^s + y^s$ for $0 < s \le 1$,inequality,1,0,0
450263,Computing $n$-th derivative $f(x)= \frac{\cos(...,"calculus,derivatives,power-series,taylor-expan...",1,0,0
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0
614772,Finding $\lim_{n\to \infty}\left((1+\frac{1}{n...,"real-analysis,limits,convergence-divergence,de...",1,0,0
1151261,How to compute $\lim\limits_{n\to\infty}\int_0...,"calculus,real-analysis,integration,limits,trig...",1,0,0
...,...,...,...,...,...
60715,NP-hardness even with perturbations,"reductions,np-hard,partitions",0,0,1
3145,Computing the distribution of a Jonckheere-Ter...,"c++,performance,r",0,0,1
18523,Representing database row as a class,"php,classes",0,0,1
19047,Basic binary search in python,"python,array,search,binary-search",0,0,1


In [69]:
test_df_with_labels = test_df  # test_df.merge(full_df.drop(columns="tags_str"), on="question", how="left")

test_df_with_labels

,question,tags_str,math,bio,code
949051,$(x + y)^s \le x^s + y^s$ for $0 < s \le 1$,inequality,1,0,0
450263,Computing $n$-th derivative $f(x)= \frac{\cos(...,"calculus,derivatives,power-series,taylor-expan...",1,0,0
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0
614772,Finding $\lim_{n\to \infty}\left((1+\frac{1}{n...,"real-analysis,limits,convergence-divergence,de...",1,0,0
1151261,How to compute $\lim\limits_{n\to\infty}\int_0...,"calculus,real-analysis,integration,limits,trig...",1,0,0
...,...,...,...,...,...
60715,NP-hardness even with perturbations,"reductions,np-hard,partitions",0,0,1
3145,Computing the distribution of a Jonckheere-Ter...,"c++,performance,r",0,0,1
18523,Representing database row as a class,"php,classes",0,0,1
19047,Basic binary search in python,"python,array,search,binary-search",0,0,1


In [70]:
test_df_with_labels["real_class"] = (test_df_with_labels["math"] * 0 +  test_df_with_labels["bio"] * 1 +
                                 test_df_with_labels["code"]
                                * 2)
test_df_with_labels

,question,tags_str,math,bio,code,real_class
949051,$(x + y)^s \le x^s + y^s$ for $0 < s \le 1$,inequality,1,0,0,0
450263,Computing $n$-th derivative $f(x)= \frac{\cos(...,"calculus,derivatives,power-series,taylor-expan...",1,0,0,0
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0,0
614772,Finding $\lim_{n\to \infty}\left((1+\frac{1}{n...,"real-analysis,limits,convergence-divergence,de...",1,0,0,0
1151261,How to compute $\lim\limits_{n\to\infty}\int_0...,"calculus,real-analysis,integration,limits,trig...",1,0,0,0
...,...,...,...,...,...,...
60715,NP-hardness even with perturbations,"reductions,np-hard,partitions",0,0,1,2
3145,Computing the distribution of a Jonckheere-Ter...,"c++,performance,r",0,0,1,2
18523,Representing database row as a class,"php,classes",0,0,1,2
19047,Basic binary search in python,"python,array,search,binary-search",0,0,1,2


In [71]:
test_df_with_labels["math_preds"] = math_model.predict_proba(test_df_with_labels["question"])[:, 1]
test_df_with_labels["bio_preds"] = bio_model.predict_proba(test_df_with_labels["question"])[:, 1]
test_df_with_labels["code_preds"] = code_model.predict_proba(test_df_with_labels["question"])[:, 1]

In [72]:
test_df_with_labels

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds
949051,$(x + y)^s \le x^s + y^s$ for $0 < s \le 1$,inequality,1,0,0,0,0.798043,0.021374,0.114312
450263,Computing $n$-th derivative $f(x)= \frac{\cos(...,"calculus,derivatives,power-series,taylor-expan...",1,0,0,0,0.997176,0.000002,0.021007
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0,0,0.774770,0.011192,0.547675
614772,Finding $\lim_{n\to \infty}\left((1+\frac{1}{n...,"real-analysis,limits,convergence-divergence,de...",1,0,0,0,0.999997,0.000043,0.000531
1151261,How to compute $\lim\limits_{n\to\infty}\int_0...,"calculus,real-analysis,integration,limits,trig...",1,0,0,0,0.999994,0.000014,0.001268
...,...,...,...,...,...,...,...,...,...
60715,NP-hardness even with perturbations,"reductions,np-hard,partitions",0,0,1,2,0.007219,0.006376,0.995322
3145,Computing the distribution of a Jonckheere-Ter...,"c++,performance,r",0,0,1,2,0.978260,0.011562,0.590170
18523,Representing database row as a class,"php,classes",0,0,1,2,0.051086,0.000261,0.993827
19047,Basic binary search in python,"python,array,search,binary-search",0,0,1,2,0.001742,0.000009,1.000000


In [73]:
import numpy as np

cols = ['math_preds', 'bio_preds', 'code_preds']

max_values = test_df_with_labels[cols].max(axis=1)
max_names = test_df_with_labels[cols].idxmax(axis=1)

class_mapping = {'math_preds': 0, 'bio_preds': 1, 'code_preds': 2}

predicted_class = max_names.map(class_mapping)

test_df_with_labels['predicted_class'] = np.where(max_values > 0.5, predicted_class, -1)
test_df_with_labels


,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
949051,$(x + y)^s \le x^s + y^s$ for $0 < s \le 1$,inequality,1,0,0,0,0.798043,0.021374,0.114312,0
450263,Computing $n$-th derivative $f(x)= \frac{\cos(...,"calculus,derivatives,power-series,taylor-expan...",1,0,0,0,0.997176,0.000002,0.021007,0
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0,0,0.774770,0.011192,0.547675,0
614772,Finding $\lim_{n\to \infty}\left((1+\frac{1}{n...,"real-analysis,limits,convergence-divergence,de...",1,0,0,0,0.999997,0.000043,0.000531,0
1151261,How to compute $\lim\limits_{n\to\infty}\int_0...,"calculus,real-analysis,integration,limits,trig...",1,0,0,0,0.999994,0.000014,0.001268,0
...,...,...,...,...,...,...,...,...,...,...
60715,NP-hardness even with perturbations,"reductions,np-hard,partitions",0,0,1,2,0.007219,0.006376,0.995322,2
3145,Computing the distribution of a Jonckheere-Ter...,"c++,performance,r",0,0,1,2,0.978260,0.011562,0.590170,0
18523,Representing database row as a class,"php,classes",0,0,1,2,0.051086,0.000261,0.993827,2
19047,Basic binary search in python,"python,array,search,binary-search",0,0,1,2,0.001742,0.000009,1.000000,2


In [74]:
from sklearn.metrics import accuracy_score


accuracy_score(test_df_with_labels["real_class"], test_df_with_labels["predicted_class"])

0.8908333333333334

## Sample for analyzing errors based on wrong label or ambiguity

In [75]:
wrong_sample = test_df_with_labels[test_df_with_labels['real_class'] != test_df_with_labels['predicted_class']].sample(n=10)

wrong_sample

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
81099,2:1 Mutual exclusion with semaphores,"concurrency,synchronization",0,0,1,2,0.747157,0.902186,0.509616,1
672304,Find when two sin graphs hit $x=0$ at the same...,"algebra-precalculus,trigonometry",1,0,0,0,0.244229,0.006471,0.714586,2
1074952,Transforming generating functions into algorit...,"combinatorics,algorithms,generating-functions,...",1,0,0,0,0.044787,0.000044,0.999987,2
64159,Denumerably many isomorphism types,first-order-logic,0,0,1,2,0.541666,0.004642,0.094417,0
63723,Binomial coefficient to approach multi-way cho...,dynamic-programming,0,0,1,2,0.777760,0.015200,0.619004,0
21607,Assigning zero to random cells in a pandas.Dat...,"python,python-3.x,random,pandas",0,0,1,2,0.084116,0.189578,0.439713,-1
1920,Summing a sequence of fractions to a limit n,"java,beginner",0,0,1,2,0.944405,0.016017,0.146631,0
73331,multithreading - duration of jobs,"operating-systems,cpu,process-scheduling,threa...",0,0,1,2,0.195671,0.023539,0.273012,-1
10450,Transpiration Process,photosynthesis,0,1,0,1,0.451809,0.131387,0.408351,-1
77749,Solving diophantine equations -- does having a...,"complexity-theory,computability,decision-probl...",0,0,1,2,0.952586,0.002017,0.229251,0


In [76]:
for question in wrong_sample['question']:
	print(question)


2:1 Mutual exclusion with semaphores
Find when two sin graphs hit $x=0$ at the same time
Transforming generating functions into algorithms that generate combinatorial objects
Denumerably many isomorphism types
Binomial coefficient to approach multi-way choices DP problem?
Assigning zero to random cells in a pandas.DataFrame
Summing a sequence of fractions to a limit n
multithreading - duration of jobs
Transpiration Process
Solving diophantine equations -- does having a bound on the size of the solution help?


## Other models winning over wrong code model test

In [77]:
test_df_with_labels[(test_df_with_labels['real_class'] == test_df_with_labels['predicted_class']) & # But other
                    # models overcame te wrong one which resulted in a correct prediction
                    (test_df_with_labels['real_class'] != 2) & # but real_class is not code
                    (test_df_with_labels['code_preds'] >= 0.5)] # would predict code

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0,0,0.774770,0.011192,0.547675,0
144932,Are these two representations correct and why?,"matrices,vectors",1,0,0,0,0.931010,0.036533,0.737204,0
1210774,Having trouble following the logic of this proof,"real-analysis,proof-explanation",1,0,0,0,0.996965,0.002035,0.679856,0
903121,Prove that the normal closure of $S \subset G ...,"abstract-algebra,group-theory",1,0,0,0,0.976765,0.000043,0.658520,0
162603,Local max and absolute max,calculus,1,0,0,0,0.723555,0.050926,0.537632,0
...,...,...,...,...,...,...,...,...,...,...
14594,Up to date B cell review,"cell-biology,immunology",0,1,0,1,0.000567,0.816077,0.751691,1
15577,Trying to understand a scientific text about n...,"neuroscience,dopamine",0,1,0,1,0.098605,0.937693,0.506601,1
23098,Is there any ethnicity difference in average p...,"human-anatomy,anatomy",0,1,0,1,0.617824,0.618955,0.586254,1
19270,Where do all the NAD+s come from?,"molecular-biology,cell-biology,plant-physiolog...",0,1,0,1,0.022285,0.851330,0.647365,1


In [78]:
test_df_with_labels[(test_df_with_labels['real_class'] != 2) &
                    (test_df_with_labels['code_preds'] >= 0.5)]

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
277583,Pigeonhole principle to prove distance between...,pigeonhole-principle,1,0,0,0,0.774770,0.011192,0.547675,0
537953,Relation between a particular codeword and pri...,"ring-theory,ideals,coding-theory,computational...",1,0,0,0,0.473070,0.022119,0.712731,2
144932,Are these two representations correct and why?,"matrices,vectors",1,0,0,0,0.931010,0.036533,0.737204,0
1210774,Having trouble following the logic of this proof,"real-analysis,proof-explanation",1,0,0,0,0.996965,0.002035,0.679856,0
903121,Prove that the normal closure of $S \subset G ...,"abstract-algebra,group-theory",1,0,0,0,0.976765,0.000043,0.658520,0
...,...,...,...,...,...,...,...,...,...,...
10002,What are some features that do not make an ind...,"human-biology,genetics,evolution",0,1,0,1,0.022741,0.678420,0.794616,2
23532,How to incorporate size data with shape in ste...,"morphology,morphometry",0,1,0,1,0.024330,0.976210,0.984897,2
3509,Merge several .ab1 sequence files into a singl...,bioinformatics,0,1,0,1,0.002908,0.808824,0.697939,1
16074,Why is free ribose not reduced to deoxyribose ...,"human-biology,biochemistry,medicine",0,1,0,1,0.027578,0.312381,0.933612,2


## Other models winning over wrong math model test

In [79]:
test_df_with_labels[(test_df_with_labels['real_class'] == test_df_with_labels['predicted_class']) &  # But other
                    # models overcame te wrong one which resulted in a correct prediction
                    (test_df_with_labels['real_class'] != 0) &  # but real_class is not math
                    (test_df_with_labels['math_preds'] >= 0.5)]  # would predict math

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
16528,Major groups of photosynthetic prokaryotes,"bacteriology,photosynthesis",0,1,0,1,0.701229,9.580015e-01,0.048030,1
18561,"Is ""muscular attachment"" synonymous to tendon?...","human-biology,human-anatomy,terminology,muscle...",0,1,0,1,0.505734,7.795571e-01,0.123744,1
20888,What is the developmental basis of the placent...,"reproduction,reproductive-biology",0,1,0,1,0.530611,8.119430e-01,0.025201,1
21037,Cyanobacteria: Gram negative or Gram positive?,"cell-biology,microbiology,bacteriology,staining",0,1,0,1,0.550860,9.398796e-01,0.036509,1
17784,What is a good approximate functional form for...,"botany,plant-physiology,theoretical-biology,st...",0,1,0,1,0.761976,9.177906e-01,0.003337,1
...,...,...,...,...,...,...,...,...,...,...
73509,Solving cycle in undirected graph in log space?,"algorithms,graphs,search-problem",0,0,1,2,0.771071,2.275545e-05,0.942359,2
9410,Prime Number Sequence generator,"python,beginner,primes",0,0,1,2,0.603842,8.460812e-04,0.885324,2
55789,"Algorithm to check whether a complete, undirec...","algorithms,graphs",0,0,1,2,0.818953,1.000000e-07,0.991200,2
46227,"Finding $k$ claws ($K_{1,3}$ bipartite graphs)...","graphs,randomized-algorithms,bipartite-matchin...",0,0,1,2,0.863193,1.895585e-05,0.976666,2


In [80]:
test_df_with_labels[(test_df_with_labels['real_class'] != 0) &
                    (test_df_with_labels['math_preds'] >= 0.5)] # all wrong predictions

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
8010,Are All Taxonomic Groups Uniquely Named?,"taxonomy,nomenclature,classification",0,1,0,1,0.891863,4.177943e-01,0.111348,0
16528,Major groups of photosynthetic prokaryotes,"bacteriology,photosynthesis",0,1,0,1,0.701229,9.580015e-01,0.048030,1
18561,"Is ""muscular attachment"" synonymous to tendon?...","human-biology,human-anatomy,terminology,muscle...",0,1,0,1,0.505734,7.795571e-01,0.123744,1
10592,What is the chemical composition / empirical f...,virus,0,1,0,1,0.807267,1.782753e-02,0.354140,0
21365,Is it possible for an organism that is polypol...,"evolution,polyploidy",0,1,0,1,0.740297,2.055627e-01,0.004711,0
...,...,...,...,...,...,...,...,...,...,...
25546,Simple linear regression of two variables,"python,statistics,machine-learning",0,0,1,2,0.972538,6.615044e-05,0.918365,0
46227,"Finding $k$ claws ($K_{1,3}$ bipartite graphs)...","graphs,randomized-algorithms,bipartite-matchin...",0,0,1,2,0.863193,1.895585e-05,0.976666,2
60708,Find internal surfaces in an oriented mesh,"algorithms,reference-request,computational-geo...",0,0,1,2,0.743869,3.713800e-01,0.090711,0
52206,Proof: Sum of weights of paths in a network,"graphs,matrices",0,0,1,2,0.586149,6.035938e-07,0.956396,2


## Other models winning over wrong bio model test

In [81]:
test_df_with_labels[(test_df_with_labels['real_class'] == test_df_with_labels['predicted_class']) &  # But other
                    # models overcame te wrong one which resulted in a correct prediction
                    (test_df_with_labels['real_class'] != 1) &  # but real_class is not bio
                    (test_df_with_labels['bio_preds'] >= 0.5)]  # would predict bio

,question,tags_str,math,bio,code,real_class,math_preds,bio_preds,code_preds,predicted_class
221523,Implied proportionality,functions,1,0,0,0,0.701987,0.693915,0.336591,0
732834,(undergraduate) Algebraic Geometry Textbook Re...,"algebraic-geometry,reference-request,book-reco...",1,0,0,0,0.866727,0.510960,0.014496,0
357892,Is this $p \to (q \to p)$ a tautology?,"logic,education",1,0,0,0,0.666703,0.603124,0.065217,0
141709,A strange integral having to do with the sopho...,"calculus,real-analysis,integration,definite-in...",1,0,0,0,0.713245,0.627442,0.014946,0
899224,How do we make fractions flip?,"derivatives,fractions",1,0,0,0,0.744816,0.712184,0.439421,0
...,...,...,...,...,...,...,...,...,...,...
22852,Early-termination foldLeft for Scala Streams,"functional-programming,scala,lazy",0,0,1,2,0.071044,0.848219,0.867074,2
6959,Reimplementation of Underscore _.throttle,"javascript,reinventing-the-wheel,timer,undersc...",0,0,1,2,0.235324,0.693915,0.802466,2
34544,Comparing two arrrays,"array,perl",0,0,1,2,0.097087,0.661275,0.938323,2
20360,depthFirstTraverse fully in C++?,"c++,c,algorithm",0,0,1,2,0.457996,0.500000,0.548915,2


In [82]:
test_df_with_labels[(test_df_with_labels['real_class'] != 1) &
## Other models winning over wrong code model test

test_df_with_labels[(test_df_with_labels['real_class'] == test_df_with_labels['predicted_class']) & # But other
                    # models overcame te wrong one which resulted in a correct prediction
                    (test_df_with_labels['real_class'] != 2) & # but real_class is not code
                    (test_df_with_labels['code_preds'] >= 0.5)] # would predict code

test_df_with_labels[(test_df_with_labels['real_class'] != 2) &
                    (test_df_with_labels['code_preds'] >= 0.5)]
                    (test_df_with_labels['bio_preds'] >= 0.5)] # all wrong predictions

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3738981596.py, line 1)

## Other models winning over right code model test

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] != test_df_with_labels['predicted_class']) & # But other
                    # models overcame the right one which resulted in a wrong prediction
                    (test_df_with_labels['real_class'] == 2) & # real_class is code
                    (test_df_with_labels['code_preds'] >= 0.5)] # would predict code

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] == 2) &
                    (test_df_with_labels['code_preds'] >= 0.5)]

## Other models winning over right math model

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] != test_df_with_labels['predicted_class']) & # But other
                    # models overcame the right one which resulted in a wrong prediction
                    (test_df_with_labels['real_class'] == 0) & # real_class is math
                    (test_df_with_labels['math_preds'] >= 0.5)] # would predict math

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] == 0) &
                    (test_df_with_labels['math_preds'] >= 0.5)]

## Other models winning over right bio model

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] != test_df_with_labels['predicted_class']) & # But other
                    # models overcame the right one which resulted in a wrong prediction
                    (test_df_with_labels['real_class'] == 1) & # real_class is bio
                    (test_df_with_labels['bio_preds'] >= 0.5)] # would predict bio

In [ ]:
test_df_with_labels[(test_df_with_labels['real_class'] == 1) &
                    (test_df_with_labels['bio_preds'] >= 0.5)]